In [1]:
import pandas as pd
import numpy as np

from memory_profiler import profile

from Pyfhel import Pyfhel, PyPtxt, PyCtxt

import torch
import torch.nn as nn

import time
import os
import sys

working_directory = "/home/falcetta/PINPOINT_Secret"

device = "cpu"
module_path = os.path.abspath(working_directory)
sys.path.append(module_path) 

from pycrcnn.net_builder.encoded_net_builder_ts import build_from_pytorch
from pycrcnn.crypto.crypto import encrypt_matrix, decrypt_matrix
from train_utils import *

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Models

In [2]:
class Square(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 2)

class Cube(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 3)
    
class Printer(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, t):
        # print(t)
        print(t.shape)
        return t


class PINPOINT_1CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_1CONV, self).__init__()

        n_kernels_1 = 32
        kernel_size_1 = 3
        out_conv_1 = n_kernels_1 * (input_size - kernel_size_1 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_1, int(out_conv_1/2)), #use without avgpool
            # nn.Linear(int(out_conv_1/2), output_horizon)   
            nn.Linear(int(out_conv_1/2), int(out_conv_1/4)),
            nn.Linear(int(out_conv_1/4), output_horizon)   
        )

    def forward(self, x):
        out = self.main(x)
        return out
    
    def __str__(self):
        return "PINPOINT_1CONV"

    
class PINPOINT_2CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_2CONV, self).__init__()
        
        n_kernels_1 = 16
        n_kernels_2 = 32
        kernel_size_1 = 5
        kernel_size_2 = 3
        
        out_conv_1 = input_size - kernel_size_1 + 1
        out_conv_2 = n_kernels_2 * (out_conv_1 - kernel_size_2 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Conv1d(in_channels=n_kernels_1, out_channels=n_kernels_2, kernel_size=kernel_size_2),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_2, int(out_conv_2/2)), #use without avgpool
            # nn.Linear(int(out_conv_2/4), output_horizon)   
            nn.Linear(int(out_conv_2/2), int(out_conv_2/4)),
            nn.Linear(int(out_conv_2/4), output_horizon)   
        )

    def forward(self, x):
        out = self.main(x)
        return out
    
    def __str__(self):
        return "PINPOINT_2CONV"

In [3]:
experiment_name = "AirlinePassengers"
seq_length = 12
forecast_horizon = 6
model_class = "PINPOINT_1CONV"

In [4]:
model = torch.load(f"{working_directory}/Experiments/models/{experiment_name}_{forecast_horizon}_{model_class}.pt")

In [5]:
model

PINPOINT_1CONV(
  (main): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,))
    (1): Square()
    (2): Flatten(start_dim=1, end_dim=-1)
    (3): Linear(in_features=320, out_features=160, bias=True)
    (4): Linear(in_features=160, out_features=80, bias=True)
    (5): Linear(in_features=80, out_features=6, bias=True)
  )
)

# Dataset

In [6]:
monthly_passengers = pd.read_csv(f"{working_directory}/data/airline-passengers.csv", parse_dates=['Month'], index_col='Month')
monthly_passengers = monthly_passengers.loc[:, 'Passengers']
monthly_passengers.index.freq = 'MS'
entire_ts = monthly_passengers
train = entire_ts.loc[:pd.Timestamp("1958-01-01")]

validation_length = int(0.05 * len(train))
validation = entire_ts.loc[train.index[-1] + entire_ts.index.freq:train.index[-1] + validation_length * entire_ts.index.freq]
test = entire_ts.loc[validation.index[-1] + entire_ts.index.freq:]

plot_name = "Monthly passengers airline"
yaxis_name = "Passengers"

train = train.append(validation)

print(train)
print(test)

Month
1949-01-01    112
1949-02-01    118
1949-03-01    132
1949-04-01    129
1949-05-01    121
             ... 
1958-02-01    318
1958-03-01    362
1958-04-01    348
1958-05-01    363
1958-06-01    435
Freq: MS, Name: Passengers, Length: 114, dtype: int64
Month
1958-07-01    491
1958-08-01    505
1958-09-01    404
1958-10-01    359
1958-11-01    310
1958-12-01    337
1959-01-01    360
1959-02-01    342
1959-03-01    406
1959-04-01    396
1959-05-01    420
1959-06-01    472
1959-07-01    548
1959-08-01    559
1959-09-01    463
1959-10-01    407
1959-11-01    362
1959-12-01    405
1960-01-01    417
1960-02-01    391
1960-03-01    419
1960-04-01    461
1960-05-01    472
1960-06-01    535
1960-07-01    622
1960-08-01    606
1960-09-01    508
1960-10-01    461
1960-11-01    390
1960-12-01    432
Freq: MS, Name: Passengers, dtype: int64


# Expected outputs

In [7]:
expected_output = []

scaler = MinMaxScaler(feature_range=(-1, 1))
_ = scaler.fit_transform(train.values.reshape(-1, 1))

_train = train.copy()
_test = test.copy()

forecast = np.array([])

for i in range(0, int(len(_test) / forecast_horizon) + 1):
    model.eval()

    inputs = _train.values.reshape(len(_train), 1)

    inputs_normalized = scaler.transform(inputs)
    inputs_normalized = torch.FloatTensor(inputs_normalized[-seq_length:]).to(device)

    predict = model(inputs_normalized.reshape(1, 1, seq_length))
    predict = scaler.inverse_transform(predict.cpu().detach().numpy())
    forecast = np.append(forecast, predict)

    for j in range(0, forecast_horizon):
        if len(_test) > 0:
            _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
            _test = _test.iloc[1:]

expected_output = pd.Series(data=forecast[:len(test)], index=test.index)

In [8]:
expected_output

Month
1958-07-01    520.413757
1958-08-01    532.351807
1958-09-01    457.260376
1958-10-01    384.845276
1958-11-01    338.960236
1958-12-01    361.566589
1959-01-01    367.861725
1959-02-01    353.837463
1959-03-01    414.583466
1959-04-01    388.217072
1959-05-01    398.853638
1959-06-01    486.320648
1959-07-01    558.161560
1959-08-01    568.390869
1959-09-01    460.886322
1959-10-01    391.912140
1959-11-01    345.749329
1959-12-01    358.987610
1960-01-01    407.683014
1960-02-01    387.016449
1960-03-01    460.594238
1960-04-01    439.257111
1960-05-01    446.987915
1960-06-01    523.505188
1960-07-01    622.956970
1960-08-01    618.396057
1960-09-01    526.692017
1960-10-01    436.182068
1960-11-01    399.500488
1960-12-01    422.201935
Freq: MS, dtype: float64

In [9]:
print(experiment_name)
print(f"MAE of model {model}, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, expected_output), 2)}")

AirlinePassengers
MAE of model PINPOINT_1CONV, forecast horizon: 6: 18.31


## Encode the models

In [10]:
HE = Pyfhel()    
HE.contextGen(p=95337867, m=4096, intDigits=16, fracDigits=128) 
HE.keyGen()
HE.relinKeyGen(30, 3)

encoded_model = build_from_pytorch(HE, model.cpu().main)

# Encrypted processing

In [11]:
decrypted_output = None

scaler = MinMaxScaler(feature_range=(-1, 1))
_ = scaler.fit_transform(train.values.reshape(-1, 1))

_train = train.copy()
_test = test.copy()

forecast = np.array([])

for i in range(0, int(len(_test) / forecast_horizon) + 1):
    inputs = _train.values.reshape(len(_train), 1)

    inputs_normalized = scaler.transform(inputs)
    inputs_normalized = inputs_normalized[-seq_length:].reshape(1, 1, seq_length)

    encrypted_input = encrypt_matrix(HE, inputs_normalized)

    for layer in encoded_model:
        encrypted_input = layer(encrypted_input)
    
    _min = HE.encodeFrac(scaler.min_[0])
    _scale = HE.encodeFrac(1.0 / scaler.scale_[0])
    
    encrypted_input = [list(map(lambda x: (x - _min)*_scale, encrypted_input[0]))]
    predict = decrypt_matrix(HE, encrypted_input)

    forecast = np.append(forecast, predict)

    for j in range(0, forecast_horizon):
        if len(_test) > 0:
            _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
            _test = _test.iloc[1:]

decrypted_output = pd.Series(data=forecast[:len(test)], index=test.index)

In [12]:
expected_output

Month
1958-07-01    520.413757
1958-08-01    532.351807
1958-09-01    457.260376
1958-10-01    384.845276
1958-11-01    338.960236
1958-12-01    361.566589
1959-01-01    367.861725
1959-02-01    353.837463
1959-03-01    414.583466
1959-04-01    388.217072
1959-05-01    398.853638
1959-06-01    486.320648
1959-07-01    558.161560
1959-08-01    568.390869
1959-09-01    460.886322
1959-10-01    391.912140
1959-11-01    345.749329
1959-12-01    358.987610
1960-01-01    407.683014
1960-02-01    387.016449
1960-03-01    460.594238
1960-04-01    439.257111
1960-05-01    446.987915
1960-06-01    523.505188
1960-07-01    622.956970
1960-08-01    618.396057
1960-09-01    526.692017
1960-10-01    436.182068
1960-11-01    399.500488
1960-12-01    422.201935
Freq: MS, dtype: float64

In [13]:
decrypted_output

Month
1958-07-01    518.890511
1958-08-01    531.603378
1958-09-01    456.511657
1958-10-01    384.072216
1958-11-01    338.748933
1958-12-01    360.762147
1959-01-01    367.427527
1959-02-01    353.057678
1959-03-01    413.054733
1959-04-01    386.683013
1959-05-01    397.336985
1959-06-01    483.233550
1959-07-01    556.577862
1959-08-01    566.886322
1959-09-01    459.356642
1959-10-01    390.384574
1959-11-01    345.558968
1959-12-01    358.191906
1960-01-01    406.859591
1960-02-01    386.246494
1960-03-01    459.055793
1960-04-01    436.136715
1960-05-01    445.477284
1960-06-01    521.948066
1960-07-01    621.467571
1960-08-01    616.906036
1960-09-01    520.506513
1960-10-01    434.622484
1960-11-01    397.953188
1960-12-01    420.652551
Freq: MS, dtype: float64

In [14]:
print(f"MAE of model {model}, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, expected_output), 2)}")
print(f"MAE of model {model} used on encrypted inputs, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, decrypted_output), 2)}")

MAE of model PINPOINT_1CONV, forecast horizon: 6: 18.31
MAE of model PINPOINT_1CONV used on encrypted inputs, forecast horizon: 6: 18.08


Difference between expected and obtained on encrypted data:

In [15]:
print(expected_output - decrypted_output)

Month
1958-07-01    1.523247
1958-08-01    0.748428
1958-09-01    0.748719
1958-10-01    0.773060
1958-11-01    0.211303
1958-12-01    0.804442
1959-01-01    0.434198
1959-02-01    0.779786
1959-03-01    1.528733
1959-04-01    1.534058
1959-05-01    1.516652
1959-06-01    3.087098
1959-07-01    1.583699
1959-08-01    1.504547
1959-09-01    1.529680
1959-10-01    1.527566
1959-11-01    0.190361
1959-12-01    0.795704
1960-01-01    0.823423
1960-02-01    0.769955
1960-03-01    1.538445
1960-04-01    3.120395
1960-05-01    1.510631
1960-06-01    1.557122
1960-07-01    1.489399
1960-08-01    1.490022
1960-09-01    6.185503
1960-10-01    1.559584
1960-11-01    1.547301
1960-12-01    1.549384
Freq: MS, dtype: float64
